# Imports

In [2]:
import os
import numpy as np
import pandas as pd
from unidecode import unidecode
from matplotlib import pyplot as plt
from matplotlib import gridspec as gridspec
import seaborn as sns
from google.cloud import storage
import math
import requests
import bs4
import re

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
pd.set_option('display.max_columns', 100)

# Funções Auxiliares

In [3]:
def get_data(bucket_name:str, imobiliarias:str = ['apolar', 'cilar'],by:str = ['date','date_diff'], dates:list = [], date_diff:int = 2):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)

    files_on_bucket = [i.name for i in bucket.list_blobs()]
    files = pd.DataFrame(files_on_bucket, columns=['name'])
    files['date'] = pd.to_datetime(files['name'].apply(lambda f: f.split(' - ')[0]))
    files['imobiliaria'] = files['name'].apply(lambda f: f.split(' - ')[-1].replace('.csv',''))

    match by:
        case 'date':
            files = files.loc[files['date'].isin(dates)]
            files = files.loc[files['imobiliaria'].isin(imobiliarias)]
        case 'date_diff':
            dates = files['date'].sort_values(ascending=False).drop_duplicates().reset_index(drop=True)[:date_diff].tolist()
            files = files.loc[files['date'].isin(dates)]
            files = files.loc[files['imobiliaria'].isin(imobiliarias)]
    
    df_full = pd.DataFrame()

    for file_name in files['name'].tolist():
        try:
            df_aux = pd.read_csv(f'gs://{bucket_name}/{file_name}')
            df_full = pd.concat([df_full, df_aux], axis = 0)
        except:
            pass

    df_full = df_full.reset_index(drop=True)

    return df_full

def barplot(title:str, 
            group:str, 
            agg:str, 
            agg_name:str, 
            data:pd.DataFrame, 
            agg_func:str, 
            figure= plt.figure, 
            title_font_size:int =10, 
            figsize=(10,5),
            subplot:plt.subplot = None, 
            grid:list = None, 
            orient:str='h',
            label=True,
            rotation_label:int = 45,
            position_label:str = 'center',
            color_label:str = 'white',
            size_label:str = 'small',
            fmt:str = '%.0f',
            sort: bool = True, 
            hue:str = None,
            stacked:bool = False):
    
    group_list = [group]
    if hue:
        group_list.append(hue)

    # group data
    aux = data[group_list + [agg]].groupby(group_list).agg(agg_func).reset_index().rename(columns={agg:agg_name})

    if sort:
        aux = aux.sort_values(agg_name, ascending=False)
        
    # plot
    if subplot:
        subplot(grid)
    else:
        figure(figsize=figsize)

    # plot configs
    plt.title(title, fontsize=title_font_size)
    plt.xticks(rotation = rotation_label)

    # figure
    if orient == 'h':
        g = sns.barplot(x = group, y = agg_name, hue = hue, dodge = not stacked, data = aux)
    elif orient == 'v':
        g = sns.barplot(y = group, x = agg_name, hue = hue, dodge = not stacked, data = aux)
    else:
        raise("Variável 'orient' informada não é válida")

    if label:
        for i in g.containers:
            g.bar_label(i, color = color_label, label_type=position_label, fontsize = size_label, fmt = fmt)
    else:
        pass

def histplot(title:str, col:str, data:pd.DataFrame, figsize=(10,5), label=True):

    plt.figure(figsize=figsize)
    plt.title(title)
    g = sns.histplot(x = col,  data = data)
    plt.xticks(rotation = 45)
    if label:
        for i in g.containers:
            g.bar_label(i, color = 'white',label_type='center')
    else:
        pass

def get_infos_curitiba():

    ## Request site
    response = requests.get('https://pt.wikipedia.org/wiki/Lista_de_bairros_de_Curitiba')

    ## beautiful soup object
    soup = bs4.BeautifulSoup(response.content, 'html.parser')

    # tabelas da wikipedia
    infos_tabela = soup.findAll('table', {'class','wikitable'})

    bairros_info_list = []

    ## para cada tabela
    for tabela in infos_tabela:

        # colunas
        columns = [i.text.replace('\n','') for i in tabela.findAll('th')]

        # valores da tabela
        regiao = ' '.join([i.text.replace('\xa0','').replace('\n','') for i in tabela.findAll('td')][0].replace('Bairros oficiais de Curitiba - Regional ','').split(' ')[:-1])
        table_values = [i.text.replace('\xa0','').replace('\n','') for i in tabela.findAll('td')][1::]
        table_values_list = [] 
        for i in range(0,len(table_values),7): 
            table_values_list.append(table_values[i:i+7])

        # preenchenco dicionário
        for b in table_values_list:

            bairros_info_dict = {}
            bairros_info_dict['Região'] = regiao
            bairros_info_dict['Bairro'] = b[0]
            bairros_info_dict['Área (km²)'] = b[1].replace(',','.')
            bairros_info_dict['Homens'] = b[2].replace(',','.')
            bairros_info_dict['Mulheres'] = b[3].replace(',','.')
            bairros_info_dict['Total'] = b[4].replace(',','.')
            bairros_info_dict['Domicilios particulares'] = b[5].replace(',','.')
            bairros_info_dict['Rendimento mensal médio porresponsáveis dos domicílios (R$)'] = b[6].replace(',','.')

            bairros_info_list.append(bairros_info_dict)
    
    return pd.DataFrame(bairros_info_list)

def busca_e_preenche_nulos(data, column, string_search):

    if isinstance(string_search, list):
        for s in string_search:
            data.loc[data[column] == 0, column] = np.nan
            data.loc[data[column].isna(), column] = data.loc[data[column].isna(), 'descricao'].apply(lambda x: 0 if isinstance(x,float) else 
                                                                                                            1 if s in unidecode(x.lower()) else 0)
    elif isinstance(string_search,str):
        data.loc[data[column] == 0, column] = np.nan
        data.loc[data[column].isna(), column] = data.loc[data[column].isna(), 'descricao'].apply(lambda x: 0 if isinstance(x,float) else 
                                                                                                            1 if string_search in unidecode(x.lower()) else 0)
    else:
        raise("Atributo 'string_search' deve ser uma lista ou uma string" )
    
    return data

def extrai_valores_string(string,substring):

    # Padronizar a expressão regular para encontrar a área total
    padrao = f'{substring} (\d+)'

    # Encontrar a área total usando regex
    area_total = re.search(padrao, string)

    if area_total:
        # Extrair o valor numérico da área total
        valor_area = area_total.group(1)
        
        # Remover vírgulas e converter para float
        valor_area = int(valor_area.replace(',', '.'))
        
    else:
        valor_area = 0
    
    return valor_area

def vagas_garagem(descricao):
    # Procura por padrões do tipo 'Garagem: [quantidade]' na descrição
    padrao = r'Garagem: (\d+)'
    resultado = re.search(padrao, descricao)

    if resultado:
        # Se encontrou, retorna a quantidade de vagas
        return int(resultado.group(1))
    else:
        # Se não encontrou ou não há informação sobre vagas, retorna 0
        return 0

def histplot_matrix(data, figsize:tuple = (10,5)):
    n_features = data.shape[1]

    rows = int(np.floor(np.sqrt(n_features)))
    cols = int(np.ceil(n_features/rows))

    grid = gridspec.GridSpec(rows,cols)

    for row in range(rows):
        for col in range(cols):
            n = (row*cols) + col
            if n >= n_features:
                break
            plt.subplot(grid[row,col])
            column_name = data.columns.tolist()[n]
            plt.title(column_name)
            data[column_name].hist(figsize=figsize)

    plt.tight_layout()

## Carregando Dados

In [4]:
bucket_name='busca-apartamentos-bucket'

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

datas = set([i.name.split(' - ')[0] for i in bucket.list_blobs()])

data = get_data(bucket_name='busca-apartamentos-bucket',by='date', dates=datas, imobiliarias=['cilar'])

In [5]:
data.shape

(0, 0)

In [ ]:
df = data.copy()

# Análise Descritiva dos Dados

In [ ]:
df.head()

,site,data_coleta,titulo,link,endereco,detalhes,aluguel,condominio,iptu,catacteristicas_imovel,detalhes_condominio,mais_detalhes_imovel,caracteristicas_imovel
0,Cilar,2024-02-21,Apartamento no 15º andar com 3 quartos,https://cilar.com.br/alugar/apartamento-no-15-...,"Rua Professor Pedro Viriato Parigot Souza, 306...","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 13.000,00","Condominio R$ 3.189,70","IPTU R$ 874,00",1ª LOCAÇÃO Apartamento de alto padrão contend...,"Piscina, Academia, Salão de festas, Playground...","Andar: 15, Ano de construção: 2021, Aptos por ...",NaN
1,Cilar,2024-02-21,Apartamento no 1º andar com 3 quartos,https://cilar.com.br/alugar/apartamento-no-1-a...,"Avenida Paraná, 111 - Cabral - Curitiba","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 12.000,00","Condominio R$ 2.348,64","IPTU R$ 814,00",Apartamento mobiliado contendo: 1 suíte com cl...,"Elevador, Garagem Coberta","Andar: 01, Ano de construção: 1999, Aptos por ...",NaN
2,Cilar,2024-02-21,Apartamento no 1º andar com 4 quartos,https://cilar.com.br/alugar/apartamento-no-1-a...,"Rua Deputado Joaquim José Pedrosa, 618 - Cabra...","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 5.000,00","Condominio R$ 1.926,04","IPTU R$ 453,00",3 salas com teto rebaixado sendo 1 com sacada ...,"Salão de festas, Playground, Quadra de esporte...","Água: sim, Andar: 01, Ano de construção: 1979,...",NaN
3,Cilar,2024-02-21,Apartamento no 11º andar com 4 quartos,https://cilar.com.br/alugar/apartamento-no-11-...,"Avenida Sete de Setembro, 4699 - Batel - Curitiba","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 4.000,00","Condominio R$ 1.350,00","IPTU R$ 445,00",1 sala para 2 ambientes; 1 sala de jantar; 1 l...,"Academia, Salão de festas, Churrasqueira, Quad...","Andar: 11, Ano de construção: 1989, Aptos por ...",NaN
4,Cilar,2024-02-21,Apartamento com 3 quartos,https://cilar.com.br/alugar/apartamento-com-3-...,"Avenida República Argentina, 1690 - Portão - C...","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 3.900,00","Condominio R$ 804,60","IPTU R$ 195,00","1 cozinha com armário, pia tampo granito, 1 me...",Garagem Coberta,"Ano de construção: 1986, Aptos por andar: 4, Á...",NaN


In [153]:
import ast
df['detalhes'] = df['detalhes'].apply(lambda x: x if pd.isna(x) else ast.literal_eval(x))
df['detalhes'] = df['detalhes'].apply(lambda x: ' '.join(x).replace('Características do imóvel ','').strip() if isinstance(x,list) else x)

df.loc[df['condominio'].str.contains('IPTU', na=False), 'iptu'] = df.loc[df['condominio'].str.contains('IPTU', na=False), 'condominio']
df.loc[df['condominio'].str.contains('IPTU', na=False), 'condominio'] = np.nan

In [154]:
def formata_valores(valores):
    return valores.str.replace('.','').apply(lambda x: x if pd.isna(x) else x.split(',')[0]).astype('float64')

def extrai_valores_string(string,substring):

    # Padronizar a expressão regular para encontrar a área total
    padrao = f'{substring} (\d+)'

    # Encontrar a área total usando regex
    area_total = re.search(padrao, string)

    if area_total:
        # Extrair o valor numérico da área total
        valor_area = area_total.group(1)
        
        # Remover vírgulas e converter para float
        valor_area = int(valor_area.replace(',', '.'))
        
    else:
        valor_area = np.nan
    
    return valor_area

In [155]:
df['aluguel'] = formata_valores(df['aluguel'].str.replace('AluguelR$',''))
df['condominio'] = formata_valores(df['condominio'].str.replace('Condominio  R$',''))
df['iptu'] = formata_valores(df['iptu'].str.replace('IPTU  R$',''))

In [156]:
## Detalhes do imóvel
df['area'] = df['detalhes'].apply(lambda x: x if pd.isna(x) else extrai_valores_string(x,'Área Total'))
df['quartos'] = df['detalhes'].apply(lambda x: x if  pd.isna(x) else extrai_valores_string(x,'Quartos'))
df['suites'] = df['detalhes'].apply(lambda x: x if  pd.isna(x) else extrai_valores_string(x,'Suítes'))
df['banheiros'] = df['detalhes'].apply(lambda x: x if  pd.isna(x) else extrai_valores_string(x,'Banheiros'))
df['andar'] = df['detalhes'].apply(lambda x: x if  pd.isna(x) else extrai_valores_string(x,'Andar'))
df['vagas_garagem'] = df['mais_detalhes_imovel'].apply(lambda x: 0 if  pd.isna(x) else extrai_valores_string(x,'Vagas de garagem:'))

In [157]:
# Localidade
df['bairro'] = df['endereco'].apply(lambda x: x if pd.isna(x) else unidecode(x.split(' - ')[-2].capitalize()))
df['cidade'] = df['endereco'].apply(lambda x: x if pd.isna(x) else unidecode(x.split(' - ')[-1].capitalize()))

# Atributos do imóvel e condomínio
df['mobiliado'] = df['catacteristicas_imovel'].apply(lambda x: x if pd.isna(x) else 'Sim' if 'mobiliado' in unidecode(x.lower()) else 'Não')
df['piscina'] = df['detalhes_condominio'].apply(lambda x: x if pd.isna(x) else 'Sim' if 'piscina' in unidecode(x.lower()) else 'Não')
df['academia'] = df['detalhes_condominio'].apply(lambda x: x if pd.isna(x) else 'Sim' if 'academia' in unidecode(x.lower()) else 'Não')
df['sacada'] = df['catacteristicas_imovel'].apply(lambda x: x if pd.isna(x) else 'Sim' if 'sacada' in unidecode(x.lower()) else 'Não')
df['churrasqueira'] = df['detalhes_condominio'].apply(lambda x: x if pd.isna(x) else 'Sim' if 'churrasqueira' in unidecode(x.lower()) else 'Não')
df['salao_de_festas'] = df['detalhes_condominio'].apply(lambda x: x if pd.isna(x) else 'Sim' if 'salao de festa' in unidecode(x.lower()) else 'Não')

site
data_coleta
titulo
link
detalhes
aluguel
condominio
iptu
area
quartos
suites
banheiros
andar
vagas_garagem
bairro


In [163]:
df['mais_detalhes_imovel'].iloc[0]

'Andar: 15, Ano de construção: 2021, Aptos por andar: 1, Área averbada: 421,62, Área de serviço: 1, Área privativa: 244.58, Área total: 421.62, BWC empregada: 1, Bonificação: 1.444,00, Conservação: Ótimo, Cozinhas: 1, Dep. Empregada: 1, Edifício: Edifício Blue Garnet, Elevadores: 2, Face do imóvel: Norte, Fitness: 1, IPTU: 874.00, Lavabos: 1, Núm de pavimentos: 24, Piscina: 1, Playground: 1, Quartos: 3, Sacadas: 2, Salão de festas: 1, Salas: 1, Suítes: 3, Tipo de garagem: Livre, Vagas de garagem: 4, Valor bruto: 14.444,00, Valor condomínio: 3189.70'

In [161]:
df.groupby('endereco').agg(data_min=('data_coleta','min'), data_max=('data_coleta','max'))

,data_min,data_max
endereco,,
"Dom João VI, 459 - Cajuru - Curitiba",2024-04-09,2024-05-07
"A SENADOR SALGADO FILHO, 5473 - UBERABA - Curitiba",2024-07-06,2024-07-13
"AL AUGUSTO STELLFELD, 1050 - CENTRO - Curitiba",2024-05-11,2024-05-21
"AL AUGUSTO STELLFELD, 1283 - CENTRO - Curitiba",2024-02-21,2024-03-19
"AL AUGUSTO STELLFELD, 295 - CENTRO - Curitiba",2024-03-07,2024-05-14
...,...,...
"dos Funcionários, 144 - Cabral - Curitiba",2024-06-01,2024-06-08
"Álvaro Andrade, 225 - Portão - Curitiba",2024-02-21,2024-03-07
"Ângelo Massignan, 820 - São Braz - Curitiba",2024-02-24,2024-06-18
